In [8]:
import csv
import os
import speech_recognition as sr
import gradio as gr
import playsound


In [9]:

# Initialize the TTS engine (optional, use Gradio audio output instead)
# engine = pyttsx3.init() # Uncomment if you prefer TTS over Gradio audio

# Load inventory from CSV
def load_inventory(file_path):
  inventory = {}
  with open(file_path, mode='r') as file:
    reader = csv.DictReader(file)
    for row in reader:
      product = row["product"]
      inventory[product] = {"quantity": int(row["quantity"]), "price": float(row["price"])}
  return inventory

# Update the file path to the resources folder
csv_file_path = os.path.join('Resources', 'mock_inventory.csv')
inventory = load_inventory(csv_file_path)


In [ ]:

# Function to convert speech to text
def speech_to_text(audio):
  recognizer = sr.Recognizer()
  with sr.Microphone() as source:
    print("Listening...")
    try:
      audio_data = recognizer.listen(source)
      text = recognizer.recognize_google(audio_data)
      return text.lower()
    except sr.UnknownValueError:
      return "I didn't quite catch that. Could you please rephrase?"
    except sr.RequestError:
      return "There seems to be an issue with the speech recognition service. Please try again later."

# Function to process commands
def process_command(command, inventory, state):
  tokens = command.split()
  if "find" in tokens and "product" in tokens:
    return "Sure, I can help you find a product. What are you looking for?", {}
  elif "check" in tokens and "stock" in tokens:
    product_name = extract_product_name(tokens)
    if product_name in inventory:
      quantity = inventory[product_name]["quantity"]
      return f"We have {quantity} {product_name}(s) in stock.", {}
    else:
      return f"I couldn't find {product_name} in our inventory. Would you like to try another product?", {}
  # Add more logic for other commands and state management
  return "I'm still learning. Can you please rephrase or try a different question?", {}

# Function to extract product name
def extract_product_name(tokens):
  for token in tokens:
    for product in inventory.keys():
      if token in product.lower():
        return product
  return ""

# Function to speak response (optional, use Gradio text output instead)
def speak(text):
  # Uncomment if using TTS engine
  # engine.say(text)
  # engine.runAndWait()
  playsound.playsound("response.mp3")
  # Assuming "response.mp3" is generated elsewhere

# Gradio interface definition
def voice_interface():
  state = {}
  while True:
    audio = speech_to_text(None)
    if audio:
      response, new_state = process_command(audio, inventory, state)
      state = new_state
      return response

interface = gr.Interface(
  fn=voice_interface,
  inputs=gr.Microphone(),
  outputs="text",
  title="Paxil's Voice Assistant",
  description="Speak into the microphone and get a voice response!",
  live=True
)

# Launch the Gradio interface
interface.launch(share=True)

print("Gradio interface launched. You can now interact with the Paxil's voice assistant.")
